In [1]:
#GPU版本 指定在CPU上运行
import os
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"

In [2]:
import tensorflow as tf
import tensorflow_hub as hub
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import re
from tensorflow import keras
import pandas as pd

print(tf.__version__)

/usr/local/lib/python2.7/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


1.8.0


# 数据预处理

# 数据加载

In [3]:
train_data=pd.read_csv("/home/xm/jupyterwork/univeral-sentence-encoder/ag_news_csv/train.csv",header=None,usecols=[1,2])
train_labels=pd.read_csv("/home/xm/jupyterwork/univeral-sentence-encoder/ag_news_csv/train.csv",header=None,usecols=[0])
test_data=pd.read_csv("/home/xm/jupyterwork/univeral-sentence-encoder/ag_news_csv/test.csv",header=None,usecols=[1,2])
test_labels=pd.read_csv("/home/xm/jupyterwork/univeral-sentence-encoder/ag_news_csv/test.csv",header=None,usecols=[0])

# titlte+description

In [4]:
#array
train_data=train_data.values
y_train=train_labels.values
test_data=test_data.values
y_test=test_labels.values

train_messages = [None]*120000
test_messages = [None]*7600

for index, data in enumerate(train_data):
    train_messages[index] = data[0]+'.'+data[1]

for index, data in enumerate(test_data):
    test_messages[index] = data[0]+'.'+data[1]

for index,data in enumerate(y_train):
    y_train[index][0]=data[0]-1
for index,data in enumerate(y_test):
    y_test[index][0]=data[0]-1

# 向量模型

In [5]:
paths="/tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47"
# Import the Universal Sentence Encoder's TF Hub module
embed = hub.Module(paths)

INFO:tensorflow:Using /tmp/tfhub_modules to cache modules.
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_0:0 from checkpoint /tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables with Embeddings_en/sharded_0
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_1:0 from checkpoint /tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables with Embeddings_en/sharded_1
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_10:0 from checkpoint /tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables with Embeddings_en/sharded_10
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_11:0 from checkpoint /tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/variables with Embeddings_en/sharded_11
INFO:tensorflow:Initialize variable module/Embeddings_en/sharded_12:0 from checkpoint /tmp/tfhub_modules/1fb57c3ffe1a38479233ee9853ddd7a8ac8a8c47/variables/varia

# encoder句子转化向量

In [6]:
# Reduce logging output.
tf.logging.set_verbosity(tf.logging.ERROR)
with tf.Session() as session:
    session.run([tf.global_variables_initializer(), tf.tables_initializer()])
    x_train = session.run(embed(train_messages))
    x_test = session.run(embed(test_messages))

# 构建模型

In [7]:
# 
from keras.layers import LSTM, Dense
model = keras.Sequential()
model.add(keras.layers.Dense(128, input_dim=512))
model.add(keras.layers.Dense(32, activation='relu'))
model.add(keras.layers.Dense(4, activation='softmax'))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 128)               65664     
_________________________________________________________________
dense_2 (Dense)              (None, 32)                4128      
_________________________________________________________________
dense_3 (Dense)              (None, 4)                 132       
Total params: 69,924
Trainable params: 69,924
Non-trainable params: 0
_________________________________________________________________


Using TensorFlow backend.


In [8]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [9]:
y_train_labels = keras.utils.to_categorical(y_train, num_classes=4)
y_train_labels

array([[0., 0., 1., 0.],
       [0., 0., 1., 0.],
       [0., 0., 1., 0.],
       ...,
       [0., 1., 0., 0.],
       [0., 1., 0., 0.],
       [0., 1., 0., 0.]])

In [10]:
model.fit(x_train, y_train_labels, epochs=30,shuffle=True)

Epoch 1/30
120000/120000 [==============================] - 32s 268us/step - loss: 0.3139 - acc: 0.89220s - loss: 0.3141 - ac
Epoch 2/30
120000/120000 [==============================] - 28s 234us/step - loss: 0.2910 - acc: 0.8986
Epoch 3/30
120000/120000 [==============================] - 28s 233us/step - loss: 0.2837 - acc: 0.8999
Epoch 4/30
120000/120000 [==============================] - 27s 228us/step - loss: 0.2792 - acc: 0.9026
Epoch 5/30
120000/120000 [==============================] - 28s 237us/step - loss: 0.2761 - acc: 0.9040
Epoch 6/30
120000/120000 [==============================] - 28s 229us/step - loss: 0.2731 - acc: 0.9057
Epoch 7/30
120000/120000 [==============================] - 28s 233us/step - loss: 0.2715 - acc: 0.9052
Epoch 8/30
120000/120000 [==============================] - 27s 225us/step - loss: 0.2707 - acc: 0.9055
Epoch 9/30
120000/120000 [==============================] - 29s 238us/step - loss: 0.2703 - acc: 0.9062
Epoch 10/30
120000/120000 [===============

In [13]:
y_test_labels = keras.utils.to_categorical(y_test, num_classes=4)
results = model.evaluate(x_test, y_test_labels)
print(results)

7600/7600 [==============================] - 1s 89us/step
[0.3460392961455019, 0.8906578947368421]


In [ ]:
predictions = model.predict(x_test)
for index,pre in enumerate(predictions):
    print('real:',y_test[index][0])
    print('   pre:',np.argmax(pre))